In [4]:
import boto3
from sentence_transformers import SentenceTransformer, util
import os
import json

In [5]:
BUCKET_NAME = "arxiv-full-dataset"
S3_PARSED_PAPERS_DIR = "parsed_papers"
PARSED_PAPERS_DIR = "../parsed_papers"

os.makedirs(PARSED_PAPERS_DIR, exist_ok=True)

s3 = boto3.client("s3")
response = s3.list_objects_v2(Bucket=BUCKET_NAME, Prefix=S3_PARSED_PAPERS_DIR)
keys = [o["Key"] for o in response.get("Contents", [])]

downloaded = 0

for key in keys:
    if downloaded >= 3:
        break

    local_path = os.path.join(PARSED_PAPERS_DIR, os.path.basename(key))

    if os.path.isdir(local_path):
        continue

    s3.download_file(BUCKET_NAME, key, local_path)
    downloaded += 1

Exception in thread Thread-18 (bg_main):
Traceback (most recent call last):
  File "/tmp/ipykernel_13469/51489788.py", line 17, in bg_main
  File "/home/ericleonen/MathCopilot/.venv/lib/python3.11/site-packages/IPython/core/display_functions.py", line 354, in update
    update_display(obj, display_id=self.display_id, **kwargs)
  File "/home/ericleonen/MathCopilot/.venv/lib/python3.11/site-packages/IPython/core/display_functions.py", line 306, in update_display
    display(obj, display_id=display_id, **kwargs)
  File "/home/ericleonen/MathCopilot/.venv/lib/python3.11/site-packages/IPython/core/display_functions.py", line 276, in display
    publish_display_data(data=obj, metadata=metadata, **kwargs)
  File "/home/ericleonen/MathCopilot/.venv/lib/python3.11/site-packages/IPython/core/display_functions.py", line 73, in publish_display_data
    display_pub.publish(
  File "/home/ericleonen/MathCopilot/.venv/lib/python3.11/site-packages/ipykernel/zmqshell.py", line 135, in publish
    msg =

In [ ]:
embedder = SentenceTransformer('math-similarity/Bert-MLM_arXiv-MP-class_zbMath')

def load_and_prepare_data(paper_files):
    """
    Loads theorem data from the specified JSON files and prepares it for embedding.
    """
    all_theorems_data = []
    for file_path in paper_files:
        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                data = json.load(f)

                global_notations   = data.get("global_notations", "")
                global_definitions = data.get("global_definitions", "")
                global_assumptions = data.get("global_assumptions", "")

                global_context_parts = []
                if global_notations:
                    global_context_parts.append(f"**Global Notations:**\n{global_notations}")
                if global_definitions:
                    global_context_parts.append(f"**Global Definitions:**\n{global_definitions}")
                if global_assumptions:
                    global_context_parts.append(f"**Global Assumptions:**\n{global_assumptions}")

                global_context = "\n\n".join(global_context_parts)
                paper_url     = data.get("url", "")
                paper_title   = data.get("title", "N/A")

                for theorem in data.get("theorems", []):
                    all_theorems_data.append({
                        "paper_title":    paper_title,
                        "paper_url":      paper_url,
                        "type":           theorem["type"],
                        "content":        theorem["content"],
                        "global_context": global_context,
                        "text_to_embed":  f"{global_context}\n\n**{theorem['type'].capitalize()}:**\n{theorem['content']}"
                    })
        except Exception:
            print("An error occurred.")

    return all_theorems_data

theorems_data = load_and_prepare_data([
    os.path.join(PARSED_PAPERS_DIR, f)
    for f in os.listdir(PARSED_PAPERS_DIR)
    if f.endswith('.json')
])

In [9]:
embedder = SentenceTransformer('math-similarity/Bert-MLM_arXiv-MP-class_zbMath')

In [11]:
embedder.encode([item['text_to_embed'] for item in theorems_data], convert_to_tensor=True)

KeyboardInterrupt: 